In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
import cv2
import torch
import os
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

In [13]:
# !python3 -m pip install --force-reinstall moviepy

In [14]:
import cv2
import torch
import pandas as pd
from tqdm.notebook import tqdm

reappearances = ""

def inference_video(model_version=7,
                    model_path = "",
                    video_path = "",
                    output_csv = "",
                    output_video = "",
                    ):
    
    global reappearances
    # === CONFIG ===

    if model_path == "":
        model_path = f'yolov5/runs/train/celeb_yolov5s{model_version}/weights/best.pt'
    if video_path == "":
        video_path = "./Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4"
    if output_csv == "":
        output_csv = f'./inference_results/s{model_version}_{video_path[:-4].replace("./","")}_detections_log.csv'
    if output_video == "":
        output_video = f'./inference_results/s{model_version}_{video_path[:-4].replace("./","")}_output_with_boxes.mp4' # This will be the video WITHOUT audio initially
    
    min_conf_score = 0.05
    max_conf_score = 0.9

    # === LOAD MODEL ===
    model = torch.hub.load('yolov5', 'custom', path=model_path, source='local')
    model.conf = min_conf_score
    # model.classes = [0, 1, 3, 4, 5, 6, 8]
    model.classes = None

    class_wise_confidence_scores = {0: 0.05,
                                    1: 0.80,
                                    2: 0.80,
                                    3: 0.80,
                                    4: 0.80,
                                    5: 0.80,
                                    6: 0.80,
                                    7: 0.80,
                                    8: 0.80}


    # === OPEN VIDEO ===
    cap = cv2.VideoCapture(video_path)
    framerate = cap.get(cv2.CAP_PROP_FPS)
    frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"frame count: {frame_count}")

    # === INIT VIDEO WRITER ===
    # Use 'mp4v' for .mp4. If you encounter issues, try 'XVID' for .avi and then convert.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, framerate, (frame_width, frame_height))

    frame_id = 0
    all_detections = []
    
    detected_classes = {0}

    print(f"Running inference on {video_path}...")

    # Use ceil to ensure progress bar accounts for partial seconds/frames
    pbar = tqdm(total=frame_count, desc="Processing Frames") 
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()

        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            cls = int(cls)

            # Draw bounding box
            label = f"{model.names[cls]} {conf:.2f}"
            color = (0, 255, 0) # Green BGR
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Apply class-wise confidence threshold
            if conf < class_wise_confidence_scores[cls]:
                continue
            
            # # Skip class 0 if it's meant to be ignored (e.g., general person)
            # if cls == 0:
            #     continue

            timestamp_seconds = frame_id / framerate
            minutes = int(timestamp_seconds // 60)
            seconds = int(timestamp_seconds % 60)
            timestamp = f"{minutes:02}:{seconds:02}" # Format as MM:SS

            if cls in detected_classes:
                reappearances += (
                    f"reappearance of {model.names[cls]} at frame {frame_id} "
                    f"with confidence {conf:.3f} at timestamp = {timestamp}.\n"
                )
                # You might want to log reappearances to a separate list/file if needed
                continue # Skip logging reappearance to all_detections

            all_detections.append({
                'frame': frame_id,
                'timestamp': timestamp, # Storing in seconds for precision
                'x1': round(x1, 2),
                'y1': round(y1, 2),
                'x2': round(x2, 2),
                'y2': round(y2, 2),
                'conf': round(conf, 3),
                'class': cls,
                'label': model.names[cls]
            })
            detected_classes.add(cls)
            print(f"{model.names[cls]} detected at frame {frame_id}; "
                f"timestamp = {timestamp}; confidence: {round(conf, 3)}")

        out.write(frame)  # Save annotated frame
        frame_id += 1
        pbar.update(1) # Update progress bar for each frame

    pbar.close()
    cap.release()
    out.release()
    print(f"Inference complete. Processed {frame_id} frames. Video saved to {output_video}")

    # === SAVE LOG ===
    df = pd.DataFrame(all_detections)
    df.to_csv(output_csv, index=False)
    print(f"Detections saved to {output_csv}")

In [15]:
# === MERGE AUDIO ===
def merge_audio_track_to_video(model_version = 7,
                               video_path = "./Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4",
                               output_video = ""):
    if output_video == "":
        output_video = f'./inference_results/s{model_version}_{video_path[:-4].replace("./","")}_output_with_boxes.mp4'
    final_output_video_with_audio = f'./inference_results/s{model_version}_output_final_with_audio.mp4' # New: This will be the final video WITH audio
    from moviepy.editor import VideoFileClip, AudioFileClip # Import moviepy
    print(f"Merging audio from {video_path} to {output_video}...")
    try:
        video_clip = VideoFileClip(output_video)
        audio_clip = AudioFileClip(video_path) # Extract audio from original video

        final_clip = video_clip.set_audio(audio_clip)
        final_clip.write_videofile(final_output_video_with_audio, codec="libx264", audio_codec="aac")

        video_clip.close()
        audio_clip.close()
        print(f"Final video with audio saved to {final_output_video_with_audio}")
    except Exception as e:
        print(f"Error merging audio: {e}")
        print("Please ensure 'ffmpeg' is installed and accessible in your system's PATH.")



In [16]:
model_version = "14"

In [17]:
video_path = "./Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4"

In [18]:
inference_video(model_version=model_version, video_path=video_path)

YOLOv5  v7.0-423-gc488fd40 Python-3.12.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


frame count: 5994
Running inference on ./Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4...


Processing Frames:   0%|          | 0/5994 [00:00<?, ?it/s]

Kate Winslet detected at frame 99; timestamp = 00:04; confidence: 0.8889999985694885
Scarlett Johansson detected at frame 144; timestamp = 00:06; confidence: 0.8169999718666077
Tom Hanks detected at frame 1076; timestamp = 00:44; confidence: 0.8270000219345093
Leonardo DiCaprio detected at frame 5265; timestamp = 03:39; confidence: 0.8069999814033508
Inference complete. Processed 5994 frames. Video saved to ./inference_results/s14_Oscars_2020_Highlights_Oscars_2009_Kate_Montage_output_with_boxes.mp4
Detections saved to ./inference_results/s14_Oscars_2020_Highlights_Oscars_2009_Kate_Montage_detections_log.csv


In [19]:
merge_audio_track_to_video(model_version=model_version, video_path=video_path)


# !ffmpeg -i "Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4" -i "output_with_boxes.mp4" -c:v copy -c:a aac -map 1:v:0 -map 0:a:0 "final_output_with_audio_direct_ffmpeg.mp4"

Merging audio from ./Oscars_2020_Highlights_Oscars_2009_Kate_Montage.mp4 to ./inference_results/s14_Oscars_2020_Highlights_Oscars_2009_Kate_Montage_output_with_boxes.mp4...
Moviepy - Building video ./inference_results/s14_output_final_with_audio.mp4.
MoviePy - Writing audio in s14_output_final_with_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./inference_results/s14_output_final_with_audio.mp4



Moviepy - Done !
Moviepy - video ready ./inference_results/s14_output_final_with_audio.mp4
Final video with audio saved to ./inference_results/s14_output_final_with_audio.mp4


In [20]:
print(reappearances)

reappearance of Kate Winslet at frame 100 with confidence 0.887 at timestamp = 00:04.
reappearance of Kate Winslet at frame 101 with confidence 0.889 at timestamp = 00:04.
reappearance of Kate Winslet at frame 102 with confidence 0.889 at timestamp = 00:04.
reappearance of Kate Winslet at frame 103 with confidence 0.877 at timestamp = 00:04.
reappearance of Kate Winslet at frame 104 with confidence 0.850 at timestamp = 00:04.
reappearance of Kate Winslet at frame 105 with confidence 0.833 at timestamp = 00:04.
reappearance of Kate Winslet at frame 106 with confidence 0.801 at timestamp = 00:04.
reappearance of Kate Winslet at frame 107 with confidence 0.853 at timestamp = 00:04.
reappearance of Kate Winslet at frame 108 with confidence 0.878 at timestamp = 00:04.
reappearance of Kate Winslet at frame 109 with confidence 0.908 at timestamp = 00:04.
reappearance of Kate Winslet at frame 110 with confidence 0.909 at timestamp = 00:04.
reappearance of Kate Winslet at frame 111 with confide